In [1]:
#import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()


 ········


In [2]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query1 = '''SELECT 
    f.title,
    f.length,
    f.rating,
    f.special_features,
    c.name AS category,
    COUNT(r.rental_id) AS total_rentals
    
FROM 
    film f
JOIN 
    film_category fc ON f.film_id = fc.film_id
JOIN 
    category c ON fc.category_id = c.category_id
LEFT JOIN 
    inventory i ON f.film_id = i.film_id
LEFT JOIN 
    rental r ON i.inventory_id = r.inventory_id
GROUP BY 
    f.film_id;
'''

data_x = pd.read_sql_query(query1, engine)
data_x.head()

data_x.shape

(1000, 6)

In [3]:
query2 = '''SELECT 
    f.title,
    MAX(IF(r.rental_date LIKE '%%2005-05%%', 1, 0)) as rented_in_may
FROM 
    film f
LEFT JOIN 
    inventory i ON f.film_id = i.film_id
LEFT JOIN 
    rental r ON i.inventory_id = r.inventory_id 
GROUP BY 
    f.title;

'''

data_y = pd.read_sql_query(query2, engine)
data_y.shape


(1000, 2)

In [4]:
data_y.head()

,title,rented_in_may
0,ACADEMY DINOSAUR,1
1,ACE GOLDFINGER,0
2,ADAPTATION HOLES,1
3,AFFAIR PREJUDICE,1
4,AFRICAN EGG,1


In [5]:
data_y.isna().sum()

title            0
rented_in_may    0
dtype: int64

In [6]:
#merging the data frames
merged_data = pd.merge(data_x, data_y, on='title')
merged_data.head()

,title,length,rating,special_features,category,total_rentals,rented_in_may
0,AMADEUS HOLY,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",Action,21,1
1,AMERICAN CIRCUS,129,R,"Commentaries,Behind the Scenes",Action,22,1
2,ANTITRUST TOMATOES,168,NC-17,"Trailers,Commentaries,Deleted Scenes",Action,10,1
3,ARK RIDGEMONT,68,NC-17,"Trailers,Commentaries,Deleted Scenes,Behind th...",Action,0,0
4,BAREFOOT MANCHURIAN,129,G,"Trailers,Commentaries",Action,18,1


In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

#getting the target 
y = merged_data['rented_in_may']

#getting the data
X = merged_data.drop('rented_in_may', axis=1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)
X_train

,title,length,rating,special_features,category,total_rentals
46,REAR TRADING,97,NC-17,"Trailers,Commentaries,Deleted Scenes",Action,17
789,PLUTO OLEANDER,84,R,Behind the Scenes,New,16
722,LUCKY FLYING,97,PG-13,"Trailers,Commentaries,Behind the Scenes",Music,13
283,OPERATION OPERATION,156,G,"Trailers,Commentaries,Deleted Scenes,Behind th...",Comedy,27
39,MINDS TRUMAN,149,PG-13,"Deleted Scenes,Behind the Scenes",Action,20
...,...,...,...,...,...,...
167,MURDER ANTITRUST,166,PG,"Trailers,Deleted Scenes,Behind the Scenes",Children,23
232,RIGHT CRANES,153,PG-13,"Trailers,Commentaries,Deleted Scenes",Classics,10
860,SUICIDES SILENCE,93,G,Deleted Scenes,Sci-Fi,0
189,ZOOLANDER FICTION,101,R,"Trailers,Deleted Scenes",Children,17


In [8]:
# Scaling data
X_train_num = X_train.select_dtypes(include = np.number)# getting the numerical values

transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_train_norm

,length,total_rentals
0,0.366906,0.515152
1,0.273381,0.484848
2,0.366906,0.393939
3,0.791367,0.818182
4,0.741007,0.606061
...,...,...
795,0.863309,0.696970
796,0.769784,0.303030
797,0.338129,0.000000
798,0.395683,0.515152


In [9]:
#removing special_features and rating because it is giving too many columns on encoding
X_train_categorical = X_train.select_dtypes(include =object)
X_train_categorical.drop(['special_features', 'title'], axis=1, inplace=True)
X_train_categorical

,rating,category
46,NC-17,Action
789,R,New
722,PG-13,Music
283,G,Comedy
39,PG-13,Action
...,...,...
167,PG,Children
232,PG-13,Classics
860,G,Sci-Fi
189,R,Children


In [10]:
#using one hot encoding on categorials
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_train_categorical)
print(encoder.categories_)


# Get the feature names generated by the encoder
column_names = encoder.get_feature_names_out()

encoded = encoder.transform(X_train_categorical).toarray()

X_train_cat = pd.DataFrame(encoded,columns=column_names)
X_train_cat

[array(['G', 'NC-17', 'PG', 'PG-13', 'R'], dtype=object), array(['Action', 'Animation', 'Children', 'Classics', 'Comedy',
       'Documentary', 'Drama', 'Family', 'Foreign', 'Games', 'Horror',
       'Music', 'New', 'Sci-Fi', 'Sports', 'Travel'], dtype=object)]


,rating_NC-17,rating_PG,rating_PG-13,rating_R,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
798,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# bgetting transformed data
X_train_transformed = np.concatenate([X_train_normalized, X_train_cat], axis=1)
X_train_transformed

array([[0.36690647, 0.51515152, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27338129, 0.48484848, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.36690647, 0.39393939, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.3381295 , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.39568345, 0.51515152, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.79136691, 0.48484848, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [13]:
#using losgistic model
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [14]:
X_train_transformed.shape

(800, 21)

In [15]:
# for numericals scaling using min max
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
# we use the transformer that was trained on the training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized, columns=X_test_num.columns)
X_test_norm
X_test_norm.shape

(200, 2)

In [17]:
# for categoricals using one hot encoding
X_test_categorical = X_test.select_dtypes(include = object)
X_test_categorical.drop(['special_features', 'title'], axis=1, inplace=True)
X_test_categorical

encoded = encoder.transform(X_test_categorical).toarray()

X_test_cat = pd.DataFrame(encoded,columns=column_names)
X_test_cat.shape

(200, 19)

In [20]:
list(zip(list(X_train_cat.columns),list(X_test_cat.columns))) #comparing if we have same columns in x_train and x_test

[('rating_NC-17', 'rating_NC-17'),
 ('rating_PG', 'rating_PG'),
 ('rating_PG-13', 'rating_PG-13'),
 ('rating_R', 'rating_R'),
 ('category_Animation', 'category_Animation'),
 ('category_Children', 'category_Children'),
 ('category_Classics', 'category_Classics'),
 ('category_Comedy', 'category_Comedy'),
 ('category_Documentary', 'category_Documentary'),
 ('category_Drama', 'category_Drama'),
 ('category_Family', 'category_Family'),
 ('category_Foreign', 'category_Foreign'),
 ('category_Games', 'category_Games'),
 ('category_Horror', 'category_Horror'),
 ('category_Music', 'category_Music'),
 ('category_New', 'category_New'),
 ('category_Sci-Fi', 'category_Sci-Fi'),
 ('category_Sports', 'category_Sports'),
 ('category_Travel', 'category_Travel')]

In [21]:
list(X_train_cat.columns)==list(X_test_cat.columns)

True

In [22]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)


In [23]:
predictions = classification.predict(X_test_transformed)
predictions
classification.score(X_test_transformed, y_test)

0.745

In [24]:
print(y_test.value_counts())

rented_in_may
1    136
0     64
Name: count, dtype: int64


In [25]:
pd.Series(predictions).value_counts()

1    161
0     39
Name: count, dtype: int64

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[ 26,  38],
       [ 13, 123]])

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

#calculating accuracy_score
print('accuracy = ', accuracy_score(y_test, predictions))

#calculating precision_score
print('precision = ', precision_score(y_test, predictions))

#calculating recall_score
print('recall = ', recall_score(y_test, predictions))

#calculating f1_score
print('f1 = ', f1_score(y_test, predictions))

accuracy =  0.745
precision =  0.7639751552795031
recall =  0.9044117647058824
f1 =  0.8282828282828283
